In [1]:
### 10 章节 数据分组 和透视表
### 透视图和可以和”透视表excel的demo.xlsx“进行效果对比

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
from pandas.tseries.offsets import Day,Hour,Minute
import sys
stdo = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout= stdo

In [3]:
def load_data(filepath):
    df = pd.read_csv(filepath,sep=',',nrows = 100,encoding = "utf-8",engine="python")
    return df

In [4]:
df = load_data('/Users/maming/Downloads/test/temp.txt')
df['c4'] = pd.to_datetime(df['c4'], format='%Y%m%d %H:%M:%S')  ## 字符串转换成datetime64
df['c6'] = df['c6'].apply(lambda x: datetime.strptime(x,'%Y%m%d %H:%M:%S'))  ## 字符串转换成datetime64
df['c7'] = df['c7'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d'))  ## 字符串转换成datetime64
df

,c1,c2,c3,c4,c5,c6,c7
0,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03
3,NaN,NaN,NaN,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [5]:
### 数据分组 相当于sql的group by ,然后对分组内的数据进行汇总
### 缺点是每一次分组,只能有一个聚合函数
print df.groupby("c5") ### 返回值是DataFrameGroupBy对象,而不是DataFrame对象
### print df.groupby(["c5","c1"]).count() ### 使用多个列进行分组
print df.groupby("c5").count() ### count会计算非NaN的个数 ---这一步比sql要好,因为他对所有的列都count了
print df.groupby("c5")["c6"].count() ### 类似sql,只对某些列求count
print df.groupby("c5")[["c6","c7"]].count() ### 类似sql,只对某些列求count
print df.groupby("c5").sum() ### 只会对number类型信息sum运算


            c1  c2  c3  c4  c6  c7
c5                                
2019-09-01   1   1   1   2   2   2
2019-09-03   1   1   1   1   1   1
2019-09-05   1   1   1   1   1   1
c5
2019-09-01    2
2019-09-03    1
2019-09-05    1
Name: c6, dtype: int64
            c6  c7
c5                
2019-09-01   2   2
2019-09-03   1   1
2019-09-05   1   1
             c1    c2    c3
c5                         
2019-09-01  3.0  12.0  22.0
2019-09-03  0.0  10.0  20.0
2019-09-05  1.0  11.0  21.0


In [6]:
### aggregate 一个分组内可以产生多个聚合函数
print df.groupby("c5").aggregate(["count","sum"]) ## 每一个列产生2个聚合数据--因为可以存储多个聚合函数,因此数list参数
print df.groupby("c5").aggregate(["count"])
print "------"
print df.groupby("c5").aggregate({"c6":"count","c2":["sum","count"]}) ### 可以为不同的列,分配不同的多个聚合函数
### 对最终结果添加0开始的索引
print df.groupby("c5").aggregate({"c6":"count","c2":["sum","count"]}).reset_index()

              c1         c2          c3      
           count  sum count   sum count   sum
c5                                           
2019-09-01     1  3.0     1  12.0     1  22.0
2019-09-03     1  0.0     1  10.0     1  20.0
2019-09-05     1  1.0     1  11.0     1  21.0
              c1    c2    c3    c4    c6    c7
           count count count count count count
c5                                            
2019-09-01     1     1     1     2     2     2
2019-09-03     1     1     1     1     1     1
2019-09-05     1     1     1     1     1     1
------
              c2          c6
             sum count count
c5                          
2019-09-01  12.0     1     2
2019-09-03  10.0     1     1
2019-09-05  11.0     1     1
           c5    c2          c6
                sum count count
0  2019-09-01  12.0     1     2
1  2019-09-03  10.0     1     1
2  2019-09-05  11.0     1     1


In [7]:
print df
### 透视表--表示对df数据集合进行二维处理
### 
pd.pivot_table(df,
               index = "c3",## 行索引
               columns = "c2",###列索引
               values = "c1",### 聚合函数处理的值
               aggfunc = ["count","sum"],## 聚合函数
               margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
               margins_name = u"汇总", ### 为ALL设置名字 ---可以输出中文
               fill_value = 0, ## 缺失值填充
               dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()

    c1    c2    c3                  c4          c5                  c6  \
0  0.0  10.0  20.0 2019-09-04 22:34:21  2019-09-03 2019-09-03 22:34:00   
1  1.0  11.0  21.0 2019-09-04 22:34:21  2019-09-05 2019-09-05 22:34:00   
2  3.0  12.0  22.0 2019-09-04 22:34:21  2019-09-01 2019-09-01 22:34:00   
3  NaN   NaN   NaN 2019-09-04 22:34:21  2019-09-01 2019-09-01 22:34:00   

          c7  
0 2019-09-03  
1 2019-09-03  
2 2019-09-03  
3 2019-09-03  


c3 count               sum               
c2      10.0 11.0 12.0 汇总 10.0 11.0 12.0   汇总
0   20     1    0    0  1    0    0    0  0.0
1   21     0    1    0  1    0    1    0  1.0
2   22     0    0    1  1    0    0    3  3.0
3   汇总     1    1    1  3    0    1    3  4.0

In [8]:
xx = pd.pivot_table(df,
               index = ["c3","c6"],## 行索引
               columns = ["c2","c5"],###列索引
               values = ["c1","c4"],### 聚合函数处理的值
               aggfunc = {"c1":["count","sum"],"c4":"count"},## 聚合函数
              ## margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
              ## margins_name = "newName", ### 为ALL设置名字
               fill_value = 0, ## 缺失值填充
               dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()

xx.to_csv(path_or_buf = "/Users/maming/Downloads/111113.csv",
                sep = ",",
                index = "false",##因为索引列没意义,所以不展示索引列,即列从0开始累加的列不展示
                encoding = "utf-8",##设置编码
                na_rep = 0### 设置缺失值
                )

In [9]:
pd.pivot_table(df,
               index = ["c3","c6"],## 行索引
               columns = ["c2"],###列索引
               values = ["c1","c4"],### 聚合函数处理的值
               aggfunc = {"c1":["count","sum"],"c6":"count"},## 聚合函数
              ## margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
              ## margins_name = "newName", ### 为ALL设置名字
               fill_value = 0, ## 缺失值填充
               dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()

c3                  c6    c1                             c6          
                             count            sum           count          
c2                            10.0 11.0 12.0 10.0 11.0 12.0  10.0 11.0 12.0
0   20.0 2019-09-03 22:34:00     1    0    0    0    0    0     1    0    0
1   21.0 2019-09-05 22:34:00     0    1    0    0    1    0     0    1    0
2   22.0 2019-09-01 22:34:00     0    0    1    0    0    3     0    0    1

In [10]:
### 透视表例子
df1 = pd.read_csv("/Users/maming/Downloads/test/dataframe_3_groupby.txt",sep='\t',nrows = 100,encoding = "utf-8",engine="python")
df1

,dt,省,市,类型,销售额,销售量
0,20190901,黑龙江,哈尔滨,直营,100,50
1,20190901,黑龙江,哈尔滨,代理商,200,100
2,20190901,黑龙江,大庆,直营,300,150
3,20190901,黑龙江,大庆,代理商,400,200
4,20190902,黑龙江,哈尔滨,直营,110,51
5,20190902,黑龙江,哈尔滨,代理商,210,101
6,20190902,黑龙江,大庆,直营,310,151
7,20190902,黑龙江,大庆,代理商,410,201
8,20190901,河北,邯郸,直营,500,350
9,20190901,河北,邯郸,代理商,600,500


In [11]:
### 默认所以number列 用mean计算均值
xx = pd.pivot_table(df1,
               index = [u"省",u"市"]## 行索引
               ##columns = ["c2","c5"],###列索引
               #values = ["c1","c4"],### 聚合函数处理的值
               #aggfunc = {"c1":["count","sum"],"c4":"count"},## 聚合函数
              ## margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
              ## margins_name = "newName", ### 为ALL设置名字
               #fill_value = 0, ## 缺失值填充
              # dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()
xx

,省,市,dt,销售量,销售额
0,河北,涿州,20190901.5,475.5,750.5
1,河北,邯郸,20190901.5,400.5,550.5
2,黑龙江,哈尔滨,20190901.5,75.5,155.0
3,黑龙江,大庆,20190901.5,175.5,355.0


In [12]:
### 行 index没啥说的
### 聚合函数正常是要从value中选择,但是也可以从index中选择,values当aggfunc不存在的key时,不显示
### 列先看聚合列、聚合函数
xx = pd.pivot_table(df1,
               index = [u"省",u"市"],## 行索引
               ##columns = ["c2","c5"],###列索引
               values = [u"销售额",u"销售量"],### 要展示的列
               aggfunc = {u"销售额":["count","sum"],u"省":"sum",u"销售量":"sum"}## 聚合函数
              ## margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
              ## margins_name = "newName", ### 为ALL设置名字
               #fill_value = 0, ## 缺失值填充
              # dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()
xx

省    市             省   销售量   销售额      
                      sum   sum count   sum
0   河北   涿州      河北河北河北河北  1902     4  3002
1   河北   邯郸      河北河北河北河北  1602     4  2202
2  黑龙江  哈尔滨  黑龙江黑龙江黑龙江黑龙江   302     4   620
3  黑龙江   大庆  黑龙江黑龙江黑龙江黑龙江   702     4  1420

In [13]:
### 行 index没啥说的
### 聚合函数正常是要从value中选择,但是也可以从index中选择,values当aggfunc不存在的key时,不显示
### 列先看聚合列、聚合函数、columns
### 一行可以比较同一天的代理商、直营数据。  也可以比较不同天的直营、代理商数据
xx = pd.pivot_table(df1,
               index = [u"省",u"市"],## 行索引
               columns = [u"dt",u"类型"],###列索引
               values = [u"销售额",u"销售量"],### 要展示的列
               aggfunc = {u"销售额":["count","sum"],u"省":"sum",u"销售量":"sum"}## 聚合函数
              ## margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
              ## margins_name = "newName", ### 为ALL设置名字
               #fill_value = 0, ## 缺失值填充
              # dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()
xx

省    市        省                         销售量                         销售额  \
                  sum                         sum                       count   
dt           20190901      20190902      20190901      20190902      20190901   
类型               代理商   直营      代理商   直营      代理商   直营      代理商   直营      代理商   
0    河北   涿州       河北   河北       河北   河北      500  450      501  451        1   
1    河北   邯郸       河北   河北       河北   河北      500  350      401  351        1   
2   黑龙江  哈尔滨      黑龙江  黑龙江      黑龙江  黑龙江      100   50      101   51        1   
3   黑龙江   大庆      黑龙江  黑龙江      黑龙江  黑龙江      200  150      201  151        1   

                                               
                       sum                     
dt    20190902    20190901      20190902       
类型直营      代理商 直营      代理商   直营      代理商   直营  
0   1        1  1      800  700      801  701  
1   1        1  1      600  500      601  501  
2   1        1  1      200  100      210  110  
3   1        1  1      400  300      410  310

In [14]:
### margins 只支持aggfunc是[]形式
xx = pd.pivot_table(df1,
               index = [u"省",u"市",u"dt"],## 行索引
               columns = [u"类型"],###列索引
               values = [u"销售额",u"销售量"],### 要展示的列
               aggfunc = ["sum","count"],## 聚合函数
               margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
               margins_name = u"汇总" ### 为ALL设置名字
               #fill_value = 0, ## 缺失值填充
              # dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()
xx

省    市        dt   sum                               count             \
                         销售量               销售额               销售量        销售额   
类型                      代理商    直营    汇总   代理商    直营    汇总   代理商 直营  汇总 代理商   
0    河北   涿州  20190901   500   450   950   800   700  1500     1  1   2   1   
1    河北   涿州  20190902   501   451   952   801   701  1502     1  1   2   1   
2    河北   邯郸  20190901   500   350   850   600   500  1100     1  1   2   1   
3    河北   邯郸  20190902   401   351   752   601   501  1102     1  1   2   1   
4   黑龙江  哈尔滨  20190901   100    50   150   200   100   300     1  1   2   1   
5   黑龙江  哈尔滨  20190902   101    51   152   210   110   320     1  1   2   1   
6   黑龙江   大庆  20190901   200   150   350   400   300   700     1  1   2   1   
7   黑龙江   大庆  20190902   201   151   352   410   310   720     1  1   2   1   
8    汇总                 2504  2004  4508  4022  3222  7244     8  8  16   8   

           
           
类型直营  汇总  
0   1   2  
1   1   2  
2   1   2  
3   1   2  
4   1   2  
5   1   2  
6   1   2  
7   1   2  
8   8  16

In [15]:
### 行 index没啥说的
### 聚合函数正常是要从value中选择,但是也可以从index中选择,values当aggfunc不存在的key时,不显示
### 列先看聚合列、聚合函数、columns
### 一行可以比较同一天的代理商、直营数据
xx = pd.pivot_table(df1,
               index = [u"省",u"市",u"dt"],## 行索引
               columns = [u"类型"],###列索引
               values = [u"销售额",u"销售量"],### 要展示的列
               aggfunc = {u"销售额":["count","sum"],u"省":"sum",u"销售量":"sum"}## 聚合函数
               ##margins = True, ## 默认行/列汇总是关闭的,默认title为ALL
              ## margins_name = "newName", ### 为ALL设置名字
               #fill_value = 0, ## 缺失值填充
              # dropna = True ## 表示把整行都是缺失值的数据删除
              ).reset_index()
xx
xx[u"销售量"]["sum"][u"代理商"]
print "-------"

xx["aaa"] = xx[u"销售量"]["sum"][u"代理商"] - xx[u"销售量"]["sum"][u"直营"]
xx

-------


省    市        dt    省       销售量        销售额               aaa
                        sum       sum      count     sum          
类型                     代理商   直营  代理商   直营   代理商 直营  代理商   直营     
0    河北   涿州  20190901   河北   河北  500  450     1  1  800  700   50
1    河北   涿州  20190902   河北   河北  501  451     1  1  801  701   50
2    河北   邯郸  20190901   河北   河北  500  350     1  1  600  500  150
3    河北   邯郸  20190902   河北   河北  401  351     1  1  601  501   50
4   黑龙江  哈尔滨  20190901  黑龙江  黑龙江  100   50     1  1  200  100   50
5   黑龙江  哈尔滨  20190902  黑龙江  黑龙江  101   51     1  1  210  110   50
6   黑龙江   大庆  20190901  黑龙江  黑龙江  200  150     1  1  400  300   50
7   黑龙江   大庆  20190902  黑龙江  黑龙江  201  151     1  1  410  310   50